# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
file = "cities.csv"

In [6]:
df = pd.read_csv(file)

In [7]:
gmaps_df = pd.DataFrame(df)

In [8]:
gmaps_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = gmaps_df[["Lat", "Lng"]]

In [7]:
humidity = gmaps_df["Humidity"].astype(float)

In [8]:
fig = gmaps.figure()

In [9]:
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)



fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
new_gmaps_df = pd.DataFrame(gmaps_df)

In [40]:
new_gmaps_df = new_gmaps_df.dropna(how='any', axis=0)

In [41]:
new_gmaps_df = new_gmaps_df[(new_gmaps_df["Max Temp"] > 85)]   



new_gmaps_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
39,39,mayor pablo lagerenza,43,PY,1558378761,33,-19.93,-60.77,91.86,14.90
47,47,port elizabeth,20,US,1558378762,46,39.31,-74.98,93.00,12.75
53,53,cabo san lucas,5,MX,1558378605,34,22.89,-109.91,91.00,11.41
73,73,virginia beach,40,US,1558378597,48,36.85,-75.98,91.00,10.29
76,76,pontes e lacerda,41,BR,1558378768,41,-15.23,-59.33,88.44,3.71
82,82,sur,66,OM,1558378769,57,22.57,59.53,88.26,14.83
83,83,araouane,77,ML,1558378769,18,18.90,-3.53,100.14,15.10
89,89,ternate,20,PH,1558378770,74,14.29,120.72,86.00,1.12
90,90,tarauaca,48,BR,1558378771,47,-8.16,-70.77,90.06,1.43


In [42]:
new_gmaps_df = new_gmaps_df[(new_gmaps_df["Wind Speed"] < 10)]


new_gmaps_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
76,76,pontes e lacerda,41,BR,1558378768,41,-15.23,-59.33,88.44,3.71
89,89,ternate,20,PH,1558378770,74,14.29,120.72,86.00,1.12
90,90,tarauaca,48,BR,1558378771,47,-8.16,-70.77,90.06,1.43
108,108,bonthe,63,SL,1558378774,70,7.53,-12.50,85.38,9.78
109,109,bara,45,NG,1558378774,25,10.41,10.60,96.54,4.99
111,111,port blair,99,IN,1558378774,71,11.67,92.75,85.20,7.18
130,130,cravo norte,100,CO,1558378778,67,6.30,-70.20,85.02,3.09
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
147,147,nagua,75,DO,1558378782,70,19.37,-69.85,89.60,6.93


In [43]:
new_gmaps_df = new_gmaps_df[(new_gmaps_df["Cloudiness"] <= 4)]

new_gmaps_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
151,151,coahuayana,1,MX,1558378783,22,18.62,-100.35,100.50,4.38
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
510,510,atar,1,MR,1558378892,12,20.52,-13.05,95.00,9.17
512,512,natchez,1,US,1558378892,74,31.56,-91.40,87.01,6.93
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = gmaps_df.reset_index(drop=True)
hotel_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16,
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37,
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05,


In [13]:
params = {
 "radius": 5000,
 "types": "hotel",
 "keyword": "hotel",
 "key": g_key
}

In [19]:
# Incomplete - I had trouble finishing resolving this part.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map